Imports

In [17]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join


#Statsmodels is a great library we can use to run regressions.
import statsmodels.api as sm
# Seaborn extends the capabilities of Matplotlib
import seaborn as sns
# Used for calculating regressions
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

**Default Values**

In [18]:
path = "../raw_data/data_stocks/"

# Start date defaults
S_YEAR = 2019
S_MONTH = 1
S_DAY = 3
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 4
E_DAY = 1
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

**Get Stock File Names in a List**

In [19]:
# listdir returns all files in the directory and isfile will return true
# if it is a file and then we store its name in our list named files
files = [x for x in listdir(path) if isfile(join(path, x))]

# Remove extension from file names
# Splitext splits the file name into 2 parts being the name and extension
# We say get all file names and then store just the name in our list named files
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['CSCO',
 'PRI',
 'HLNE',
 'HUBS',
 'SBGI',
 'UFCS',
 'GFED',
 'AIRT',
 'SITC',
 'RHE',
 'ISSC',
 'TRNS',
 'AEIS',
 'UAL',
 'TROW',
 'ISRG',
 'EGP',
 'EFC',
 'EEX',
 'NVR',
 'PLYM',
 'FEIM',
 'WOR',
 'PRGO',
 'LECO',
 'RPT',
 'VCYT',
 'COHN',
 'PLOW',
 'UVE',
 'ORRF',
 'OESX',
 'SIBN',
 'NOG',
 'BXC',
 'CHCO',
 'LBTYK',
 'SPOK',
 'TPR',
 'DVN',
 'MGEE',
 'AMSC',
 'SVT',
 'SCYX',
 'MN',
 'LOVE',
 'ASYS',
 'INSE',
 'HAFC',
 'CR',
 'ADTN',
 'JVA',
 'JNCE',
 'OFLX',
 'DRRX',
 'BV',
 'AM',
 'W',
 'CE',
 'ASTC',
 'MRO',
 'HOV',
 'NBIX',
 'EVTC',
 'VYGR',
 'BA',
 'VRTX',
 'AFG',
 'GILD',
 'WW',
 'MDB',
 'MGY',
 'EVBN',
 'SLM',
 'NLSN',
 'AMKR',
 'VPG',
 'THC',
 'FCFS',
 'VRAY',
 'LOOP',
 'TAYD',
 'MDU',
 'XELA',
 'TDW',
 'WSTG',
 'CEI',
 'CFR',
 'PFMT',
 'DCO',
 'TVTY',
 'EQIX',
 'ZG',
 'HVT',
 'MKL',
 'QUIK',
 'HUN',
 'MBOT',
 'TER',
 'CULP',
 'SMBK',
 'JOB',
 'GPMT',
 'PODD',
 'MDT',
 'ARC',
 'ATGE',
 'IPAR',
 'DLA',
 'STAR',
 'SND',
 'AXSM',
 'BOOT',
 'MDC',
 'AMSWA',
 'FORM',
 'CPIX',
 'V

**Create a DataFrame from our List**

In [20]:
stock_df = pd.DataFrame(tickers,columns=['Ticker'])
stock_df

,Ticker
0,CSCO
1,PRI
2,HLNE
3,HUBS
4,SBGI
...,...
2891,PPL
2892,OBCI
2893,ROKU
2894,MTDR


**Function that Return a DataFrame from a CSV**

In [21]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(folder, ticker):
    try:
        df = pd.read_csv(folder + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        print("File Doesn't Exist")
    else:
        return df

**Function that Saves Dataframe to CSV**

In [22]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(path + ticker + '.csv')

**Delete Unamed Columns in CSV Files**

In [23]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

**Add Daily Return to Dataframe**

In [24]:
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    df.to_csv(path + ticker + '.csv')
    return df  

**Returns Return on Investment over Time**

Return on Investment is the return you receive from your investment
This amount does not include your initial investment

If you invest 100 and have 200 after 5 years
ROI = end value (200) - Initial value (100) / Initial value  = 1
your new total is Initial Investment + 1* Initial Investment = 200

In [25]:
# Return on Investment is the return you received from your investment
# This amount does not include your initial investment
# If you invest 100 and have 200 after 5 years
# ROI = End Value (200) - Initial Value (100) / Inital Value = 1
# Your new total is Inital Investment + 1 * Inital Investment = 200

def get_roi_defined_time(df):
    # Set as a datetime
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_DATE_STR]['Adj Close'][0]
    print("Initial Price :", start_val)
    
    # ----- I CHANGED THIS AFTER THE VIDEO -----
    
    end_val = df[df['Date'] == E_DATE_STR]['Adj Close']
    print(end_val.item())
    print("Final Price :", end_val.item())
    
    # ----- END OF VIDEO CHANGES -----
    
    # Calculate return on investment
    roi = (end_val - start_val) / start_val

    # Return the total return between 2 dates
    return roi

**Get Coefficient of Variation**

In [26]:
# Receives the dataframe with the Adj Close data and returns the coefficient of variation
def get_cov(stock_df):
    mean = stock_df['Adj Close'].mean()
    sd = stock_df['Adj Close'].std()
    cov = sd / mean
    return cov

**Test Functions**

In [28]:
# Create a backup for all original stock data

# Get our 1st ticker
tickers[0]

# Get a dataframe for that ticker
stock_a = get_df_from_csv(path, tickers[0])
stock_a

# Add daily return to this dataframe
add_daily_return_to_df(stock_a, tickers[0])
stock_a

# Delete unnamed columns in dataframe
stock_a = delete_unnamed_cols(stock_a)
stock_a


# Save cleaned dataframe to csv
save_dataframe_to_csv(stock_a, tickers[0])

**Add Daily Returns & Clean Up All Files**

In [29]:
# # Create a backup for all original stock data

# # Cycle through all tickers
# for ticker in tickers:
#     print("Working on :", ticker)
    
#     # Get a dataframe for that ticker
#     stock_df = get_df_from_csv(ticker)
    
#     # Add daily return to this dataframe
#     add_daily_return_to_df(stock_df, ticker)
    
#     # Delete unnamed columns in dataframe
#     stock_df = delete_unnamed_cols(stock_df)
    
#     # Save cleaned dataframe to csv
#     save_dataframe_to_csv(stock_df, ticker)

Teste

In [30]:
ticker_teste = tickers[0]
ticker_teste

'CSCO'

In [31]:
a_df = get_df_from_csv(path, ticker_teste)
a_df

,Adj Close,daily_return
Date,,
2019-01-02,39.001888,NaN
2019-01-03,37.583458,-0.036368
2019-01-04,39.276409,0.045045
2019-01-07,39.541798,0.006757
2019-01-08,39.862080,0.008100
...,...,...
2022-03-28,55.139999,0.003093
2022-03-29,55.669998,0.009612
2022-03-30,56.049999,0.006826


In [32]:
a_df2 = a_df.asfreq('d')
a_df2

,Adj Close,daily_return
Date,,
2019-01-02,39.001888,NaN
2019-01-03,37.583458,-0.036368
2019-01-04,39.276409,0.045045
2019-01-05,NaN,NaN
2019-01-06,NaN,NaN
...,...,...
2022-03-28,55.139999,0.003093
2022-03-29,55.669998,0.009612
2022-03-30,56.049999,0.006826


In [33]:
a_df2.index
a_df2

,Adj Close,daily_return
Date,,
2019-01-02,39.001888,NaN
2019-01-03,37.583458,-0.036368
2019-01-04,39.276409,0.045045
2019-01-05,NaN,NaN
2019-01-06,NaN,NaN
...,...,...
2022-03-28,55.139999,0.003093
2022-03-29,55.669998,0.009612
2022-03-30,56.049999,0.006826


In [34]:
a_df3 = a_df2.fillna(method='ffill')
a_df3

,Adj Close,daily_return
Date,,
2019-01-02,39.001888,NaN
2019-01-03,37.583458,-0.036368
2019-01-04,39.276409,0.045045
2019-01-05,39.276409,0.045045
2019-01-06,39.276409,0.045045
...,...,...
2022-03-28,55.139999,0.003093
2022-03-29,55.669998,0.009612
2022-03-30,56.049999,0.006826


In [35]:
a_df3.drop(a_df.columns[a_df.columns.str.contains('unnamed',case = False)],
          axis = 1, inplace = True)

In [36]:
a_df4 = a_df3.drop(['daily_return'], axis=1)
a_df4

,Adj Close
Date,
2019-01-02,39.001888
2019-01-03,37.583458
2019-01-04,39.276409
2019-01-05,39.276409
2019-01-06,39.276409
...,...
2022-03-28,55.139999
2022-03-29,55.669998
2022-03-30,56.049999


In [37]:
lags = ar_select_order(a_df4, maxlag=30)
lags

In [39]:
model = AutoReg(a_df4['Adj Close'], lags.ar_lags)
model_fit = model.fit()

In [41]:
# Define training and testing area
print("Length :",len(a_df)) # 1186 observations
train_df = a_df.iloc[50:656] # 80% minus 1st 50
test_df = a_df.iloc[656:] # Last 20%

Length : 820


In [42]:
# Define training model for 500 days (Play with Number & Test)
    # and White's covariance estimator
train_model = AutoReg(a_df4['Adj Close'], 500).fit(cov_type="HC0")

In [43]:
# Define start and end for prediction 
start = len(train_df)
end = len(train_df) + len(test_df) - 1

In [44]:
prediction = train_model.predict(start=start, end=end, dynamic=True)

In [45]:
# Predict 160 days into the future
forecast = train_model.predict(start=end, end=end+60, dynamic=True)

In [48]:
# Get starting price of prediction
s_price = forecast.head(1).iloc[0]

    # Get the last price of prediction
e_price = forecast.iloc[-1]

    # Get return over prediction
result = (e_price - s_price) / s_price
result

0.013505336699306118

In [51]:
def calc_projected_roi(ticker):
    a_df = get_df_from_csv(path, ticker)

    a_df = a_df.asfreq('d') # Change frequency to day
    a_df.index # Check frequency
    a_df = a_df.fillna(method='ffill') # Fill missing values

    # Delete unnamed column
    a_df.drop(a_df.columns[a_df.columns.str.contains('unnamed',case = False)],
          axis = 1, inplace = True)

    # Delete daily return column
    a_df = a_df.drop(['daily_return'], axis=1)
    
    # Figure out optimum lags which will be 1 or 2 for this data set
    lags = ar_select_order(a_df, maxlag=30)

    # Create our model using whole data set
    model = AutoReg(a_df['Adj Close'], lags.ar_lags)
    model_fit = model.fit()

    # Define training and testing area
    print("Length :",len(a_df)) # 1712 observations
    train_df = a_df.iloc[50:656] # 80% minus 1st 50
    test_df = a_df.iloc[656:] # Last 20%

    # Define training model for 500 days (Play with Number & Test)
    # and White's covariance estimator
    train_model = AutoReg(a_df['Adj Close'], 500).fit(cov_type="HC0")

    # Define start and end for prediction 
    start = len(train_df)
    end = len(train_df) + len(test_df) - 1

    prediction = train_model.predict(start=start, end=end, dynamic=True)

    # Predict 160 days into the future
    forecast = train_model.predict(start=end, end=end+60, dynamic=True)

    # Get starting price of prediction
    s_price = forecast.head(1).iloc[0]

    # Get the last price of prediction
    e_price = forecast.iloc[-1]

    # Get return over prediction
    return (e_price - s_price) / s_price

In [52]:
def get_proj_rois():
    # Will hold all tickers & stock rois
    ticker = []
    roi = []
    
    for x in tickers:
        print("Working on :", x)
        try:
            the_roi = calc_projected_roi(x)
        except Exception as ex:
            print("Stock Data Corrupted")
        else:
            ticker.append(x)
            print("ROI :", the_roi)
            roi.append(the_roi)
        
    return pd.DataFrame({'Ticker':ticker, 'ROI':roi})

In [53]:
proj_roi_df = get_proj_rois()
proj_roi_df

Working on : CSCO
Length : 1186
ROI : 0.0056296062876092555
Working on : PRI
Length : 1186
ROI : 0.034297466199485656
Working on : HLNE
Length : 1186
ROI : -0.026147471399634747
Working on : HUBS
Length : 1186
ROI : 0.14312639637453475
Working on : SBGI
Length : 1186
ROI : 0.156817511158344
Working on : UFCS
Length : 1186
ROI : 0.23730438543249088
Working on : GFED
Length : 1186
ROI : -0.060144449227453343
Working on : AIRT
Length : 1186
ROI : -0.16307272631893083
Working on : SITC
Length : 1186
ROI : 0.026718789863708705
Working on : RHE
Length : 1186
ROI : 7.626173916015179
Working on : ISSC
Length : 1186
ROI : 0.16227476308858296
Working on : TRNS
Length : 1186
ROI : 0.08566044130240427
Working on : AEIS
Length : 1186
ROI : -0.28959097788066956
Working on : UAL
Length : 1186
ROI : -0.0487311026404347
Working on : TROW
Length : 1186
ROI : 0.005806124633118644
Working on : ISRG
Length : 1186
ROI : -0.06528064255249502
Working on : EGP
Length : 1186
ROI : 0.20980823280856245
Working on

ROI : 0.2432297883949168
Working on : PPSI
Length : 1186
ROI : -0.1616023902837185
Working on : LPTH
Length : 1186
ROI : -0.8127357551146599
Working on : VCTR
Length : 1186
ROI : -0.2939119652045135
Working on : ENPH
Length : 1186
ROI : 1.08946409252345
Working on : NCR
Length : 1186
ROI : 0.012718867918930604
Working on : ITCI
Length : 1186
ROI : 0.0036967234342882896
Working on : KNSA
Length : 1186
ROI : 0.12216119375728017
Working on : BEAT
Length : 142
Stock Data Corrupted
Working on : MHLD
Length : 1186
ROI : -0.05561513168842857
Working on : APEI
Length : 1186
ROI : 0.16256789348347486
Working on : UTL
Length : 1186
ROI : 0.08706391348880779
Working on : MARK
Length : 1186
ROI : -1.0802716434935244
Working on : UBA
Length : 1186
ROI : -0.021073303852407323
Working on : BPOP
Length : 1186
ROI : 0.03848477546195286
Working on : XTNT
Length : 1186
ROI : -2.190569644925378
Working on : EYES
Length : 1186
ROI : 3.9131753382372425
Working on : ALCO
Length : 1186
ROI : 0.082654645538075

ROI : 0.0729164709321415
Working on : LJPC
Length : 1186
ROI : -0.1305011801982271
Working on : ARW
Length : 1186
ROI : -0.048892029552217546
Working on : SYY
Length : 1186
ROI : 0.03231760700472298
Working on : GENC
Length : 1186
ROI : -0.0828683331665281
Working on : CLXT
Length : 1186
ROI : -0.27589274891501525
Working on : CYAN
Length : 1186
ROI : 0.2801026331858047
Working on : MSI
Length : 1186
ROI : 0.03522864970652607
Working on : BC
Length : 1186
ROI : 0.14256501053485204
Working on : CBTX
Length : 1186
ROI : 0.0524164063120242
Working on : AX
Length : 1186
ROI : -0.16235722557299306
Working on : FCX
Length : 1186
ROI : 0.031821074974752575
Working on : WASH
Length : 1186
ROI : 0.0003074463247945064
Working on : HLX
Length : 1186
ROI : 0.33897463075246964
Working on : FBK
Length : 1186
ROI : -0.004364926800969995
Working on : SYN
Length : 1186
ROI : 0.3004619876426851
Working on : B
Length : 1186
ROI : -0.020894680511012842
Working on : PARR
Length : 1186
ROI : 0.0269156751683

ROI : -0.3809188730290442
Working on : AGE
Length : 1186
ROI : 0.4568558581393982
Working on : CC
Length : 1186
ROI : -0.323016643087125
Working on : ATRC
Length : 1186
ROI : -0.37072214101539475
Working on : UEIC
Length : 1186
ROI : -0.009025851951655199
Working on : VLGEA
Length : 1186
ROI : 0.13299222594719423
Working on : AGR
Length : 1186
ROI : 0.020540094420644854
Working on : ADI
Length : 1186
ROI : 0.021618068970609756
Working on : F
Length : 1186
ROI : -0.10337498555388266
Working on : HOG
Length : 1186
ROI : -0.08793481685478577
Working on : TZOO
Length : 1186
ROI : -0.25197363568532144
Working on : ADBE
Length : 1186
ROI : -0.008662203602380762
Working on : CYTK
Length : 1186
ROI : 0.010615318802949495
Working on : HMHC
Length : 1186
ROI : 0.24925683285268505
Working on : VNRX
Length : 1186
ROI : 0.027866585556762715
Working on : ZDGE
Length : 1186
ROI : -2.0120325738895084
Working on : CHH
Length : 1186
ROI : -0.03726806551452467
Working on : THR
Length : 1186
ROI : 0.05469

ROI : -0.06299743147662364
Working on : AEHR
Length : 1186
ROI : 0.35578229256582805
Working on : KURA
Length : 1186
ROI : 0.37178848226517663
Working on : LSXMK
Length : 1186
ROI : -0.12365218868829897
Working on : VOXX
Length : 1186
ROI : 0.16519314939927837
Working on : MDLZ
Length : 1186
ROI : 0.004026804772380533
Working on : IDT
Length : 1186
ROI : -0.3855598839492205
Working on : NBR
Length : 1186
ROI : 0.4513872801153789
Working on : BKNG
Length : 1186
ROI : -0.06307312601626969
Working on : CHMA
Length : 947
Stock Data Corrupted
Working on : WOW
Length : 1186
ROI : -0.2703845284904231
Working on : WLL
Length : 1186
ROI : 0.24954104419518108
Working on : ESP
Length : 1186
ROI : 0.12312402388899148
Working on : SBSI
Length : 1186
ROI : -0.026928703785245207
Working on : CTRE
Length : 1186
ROI : 0.1428007792832761
Working on : WCC
Length : 1186
ROI : 0.06797630816574811
Working on : PLAN
Length : 1186
ROI : 0.557939583611184
Working on : UVSP
Length : 1186
ROI : -0.10090679900616

...

ROI : -0.05764683417720667
Working on : CHS
Length : 1186
ROI : -0.19377321166660624
Working on : TKR
Length : 1186
ROI : 0.09372876215747783
Working on : KFFB
Length : 1186
ROI : -0.04224694970604506
Working on : SLP
Length : 1186
ROI : 0.4930084406015219
Working on : SNX
Length : 1186
ROI : -0.12464991608357946
Working on : CHD
Length : 1186
ROI : -0.0012367711559101667
Working on : MNTX
Length : 1186
ROI : 0.2791069430426935
Working on : SLG
Length : 1186
ROI : 0.14115439592745332
Working on : HTA
Length : 1186
ROI : 0.016118957849349747
Working on : NEWR
Length : 1186
ROI : -0.1984156959151546
Working on : BANC
Length : 1186
ROI : -0.021901168067303113
Working on : ZM
Length : 1080
ROI : 0.20980548641865815
Working on : TEN
Length : 1186
ROI : 0.14200575949677477
Working on : ESPR
Length : 1186
ROI : 0.2531188457784244
Working on : INFN
Length : 1186
ROI : -0.13097743949743323
Working on : AMED
Length : 1186
ROI : 0.20407546314625144
Working on : STOR
Length : 1186
ROI : -0.0327637

ROI : 0.13635952883876004
Working on : UHAL
Length : 1186
ROI : -0.12209269815385179
Working on : VOYA
Length : 1186
ROI : 0.03521245205630505
Working on : RDI
Length : 1186
ROI : -0.1423690175753832
Working on : RGR
Length : 1186
ROI : 0.14987512381309673
Working on : NTWK
Length : 1186
ROI : -0.26313808246696235
Working on : CASS
Length : 1186
ROI : -0.04309510401958719
Working on : APRN
Length : 1186
ROI : -0.32615799294615
Working on : RUSHA
Length : 1186
ROI : 0.057491092847834664
Working on : ISDR
Length : 1186
ROI : 0.28361920927491907
Working on : WMB
Length : 1186
ROI : 0.014683287769689163
Working on : KLIC
Length : 1186
ROI : 0.11631538009362635
Working on : XOMA
Length : 1186
ROI : 0.8872747549649721
Working on : EQR
Length : 1186
ROI : 0.044935994391234566
Working on : WKHS
Length : 1186
ROI : -3.422987949236011
Working on : PBCT
Length : 1186
ROI : -0.19685136402940948
Working on : CFBK
Length : 1186
ROI : -0.08066101752225789
Working on : NLS
Length : 1186
ROI : 2.395689

ROI : 0.11905573599937602
Working on : NATI
Length : 1186
ROI : -0.09202281976935768
Working on : KRG
Length : 1186
ROI : 0.03405607911369233
Working on : TEUM
Length : 1186
ROI : 0.005818502557247879
Working on : PEN
Length : 1186
ROI : -0.054896247900573014
Working on : NCMI
Length : 1186
ROI : -0.21602162055915802
Working on : CTRN
Length : 1186
ROI : -1.4514231106722169
Working on : GSKY
Length : 1186
ROI : -0.00868382627944657
Working on : HIFS
Length : 1186
ROI : -0.06812874487666641
Working on : WMT
Length : 1186
ROI : 0.030831823167244317
Working on : SINT
Length : 1186
ROI : -0.6560479839236276
Working on : LITE
Length : 1186
ROI : -0.06682455954496307
Working on : RPM
Length : 1186
ROI : 0.003663722149114901
Working on : RRR
Length : 1186
ROI : -0.1261114249214663
Working on : NTAP
Length : 1186
ROI : -0.24777643738370306
Working on : NBY
Length : 1186
ROI : -0.34612270409134316
Working on : SILK
Length : 1094
ROI : 0.33870436425995076
Working on : BTN
Length : 1186
ROI : -0.

ROI : 0.13898622582221468
Working on : Z
Length : 1186
ROI : 0.36820973626379466
Working on : MERC
Length : 1186
ROI : 0.08995676366385925
Working on : STWD
Length : 1186
ROI : -0.005871507200436072
Working on : BL
Length : 1186
ROI : 0.03336331238654417
Working on : M
Length : 1186
ROI : -0.07249840543449967
Working on : ACRE
Length : 1186
ROI : 0.03261696381269147
Working on : NWPX
Length : 1186
ROI : -0.09123469243833149
Working on : OSTK
Length : 1186
ROI : -0.5605621812502243
Working on : LMAT
Length : 1186
ROI : 0.292149971901596
Working on : TPH
Length : 1186
ROI : -0.05789949284144215
Working on : NBEV
Length : 1186
ROI : 0.08096381658953156
Working on : FLO
Length : 1186
ROI : -0.05294017814895441
Working on : AIR
Length : 1186
ROI : 0.20222962055036423
Working on : CRM
Length : 1186
ROI : -0.09246621271339142
Working on : CPE
Length : 1186
ROI : 0.2231166251272716
Working on : RYTM
Length : 1186
ROI : 2.860459910314356
Working on : NX
Length : 1186
ROI : 0.05684213174271495
W

,Ticker,ROI
0,CSCO,0.005630
1,PRI,0.034297
2,HLNE,-0.026147
3,HUBS,0.143126
4,SBGI,0.156818
...,...,...
2876,PPL,-0.008471
2877,OBCI,0.044087
2878,ROKU,-0.675735
2879,MTDR,0.406497


In [56]:
proj_roi_df.head()

,Ticker,ROI
0,CSCO,0.005630
1,PRI,0.034297
2,HLNE,-0.026147
3,HUBS,0.143126
4,SBGI,0.156818


In [57]:
proj_roi_df.sort_values(by=['ROI'], ascending=False)[0:10]

,Ticker,ROI
1504,RVI,43.975846
2261,CPSH,16.622310
463,MRIN,8.618036
1987,ENG,8.031697
9,RHE,7.626174
157,EYES,3.913175
2264,CRIS,3.552601
166,ARDX,3.349317
2196,MVIS,2.875123
2841,RYTM,2.860460
